In [1]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import PyPDF2,urllib.request,nltk,textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
import pdfplumber
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gauthammallipeddi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pdf=pdfplumber.open('CSE.pdf')
pdfFileObj = open('CSE.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
punctuations=["(",")",".",",","[","]","-",":",";","’"]  
text=""
for i in range(0,pdfReader.numPages):
    page=pdf.pages[i]
    text+=(page.extract_text())
    text=text.lower()
list_id = [i.start() for i in re.finditer("unit-", text)]
units=[]
for x in range(0,len(list_id)-1):
    start=list_id[x]
    end=list_id[x+1]
    units.append(text[start:end])
units.append(text[list_id[x]:int(len(text))])


subjects=[[]]
j=6
i=0
while(j<len(units)):
    subjects.append(units[i:j])
    i=j
    j=j+6

In [3]:
finalsub=subjects[0:9]

In [4]:
del(finalsub[0])

In [5]:
for sub in finalsub:
    id=sub[5].find("text books")
    temp=sub[5][:id]
    sub[5]=temp
    i=i+5

In [6]:
for sub in finalsub:
       for un in sub:
            un=un.lower()

In [7]:
pagefirst=pdf.pages[0]
page0=pagefirst.extract_text()
page0=page0.lower()
tokens=word_tokenize(page0)
stop_words=stopwords.words('english')
keywordspage0=[word for word in tokens if not word in stop_words and not word in punctuations]

In [8]:
subcode_list=list()
check=['19bs1mt01','19BS1PH02','19ES1CS01','19ES1EE01','19ES3ME02','19BS1MT04','19BS1MT05','19BS1CH01']
for i in range(0,len(check)):
    check[i]=check[i].lower()
for idx,token in enumerate(tokens):
    if(token.startswith(tuple(check))and idx<(len(tokens)-1)):
        name=token
        subcode_list.append(name)

In [9]:
dataset={}
i=0
for code in subcode_list:
    print(code)
    dataset.update({code:finalsub[i]})
    i=i+1

19bs1mt01
19bs1ph02
19es1cs01
19es1ee01
19es3me02
19bs1mt04
19bs1mt05
19bs1ch01


In [10]:
for  code in subcode_list:
    temp=[]
    for unit_temp in dataset[code]:
        unit_temp=word_tokenize(unit_temp)
#         temp=[word for word in unit_temp if not word in stop_words and not word in punctuations]
        temp.append(unit_temp)
    dataset[code]=temp

In [11]:
for code in subcode_list:
    tempt=[[]]
    for item in dataset[code]:
        temp=[word for word in item if not word in stop_words and not word in punctuations]
        tempt.append(temp)
    dataset[code]=tempt

In [12]:
import cv2 
import pytesseract
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
import nltk
from nltk.metrics.distance  import edit_distance
from nltk.corpus import stopwords
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os

In [13]:
def pdf_word_extract(open_file):
    images = convert_from_path(open_file)

    for i in range(len(images)):
        images[i].save('page'+ str(i) +'.jpg', 'JPEG')

    lister=[]
    answer=[]
    for i in range(0, len(images)):
        filename = "page"+str(i)+".jpg"

        text = str(((pytesseract.image_to_string(PIL.Image.open(filename)))))
        text = text.replace('-\n', '')    

        lister=word_tokenize(text)
        answer=answer+lister


    solution=[]
    punc=["(",")",".",",","[","]","-",":",";","’"," ","",''] 
    stop_words=stopwords.words('english')
    solution=[word.lower() for word in answer if not word.lower() in stop_words and not word in punc]
    
#     import os
#     os.remove(file) for file in os.listdir('path/to/directory') if file.endswith('.jpg')
    return solution

In [14]:
def idf(pdfwords):
    freq={}
    for word in pdfwords:
        if (word in freq):
            freq[word] += 1
        else:
            freq[word] = 1

    return freq

def findSubAndUnit(subcodelist,data,userPdf,idf):
    count={}
    
    for code in subcodelist:
        unitNumber=0
        listOfPair=[[]]
        for unit in data[code]:
            freq=0
            for word in unit:
                if word in userPdf:
                    freq=freq+idf[word]
            tl=[unitNumber,freq]
            listOfPair.append(tl)
            unitNumber=unitNumber+1
        count[code]=listOfPair
    codereq=[]
    unitreq=0
    maxFreq=0
    for key in count:
        for i in range(0,len(count[key])):
            j=1
            tu=1
            for val in count[key][i]:
                if(j==1):
                    tu=val
                if(j==2):
                    if val>maxFreq:
                        maxFreq=val
                        codereq=key
                        unitreq=tu
                j=j+1
    rli=[]
    rli.append(codereq)
    rli.append(unitreq)
    return rli

In [15]:
def FindSubName(code):
    sdict={'19bs1mt01':'Calculus for Engineers','19bs1ph02':'Engineering Physics','19es1cs01':'Programming through C','19es1ee01':'Basics of Electrical Energy for Engineers','19es3me02':'Engineering Drawing','19bs1mt04':'Linear Algebra and Advanced Calculus','19bs1mt05':'Statistical Methods for Data Analysis','19bs1ch01':'Engineering Chemistry'}
    return sdict[code]

In [16]:
import os
import shutil
def path_move(open_file,direct):
    directory=direct
    parent_direct= '/Users/gauthammallipeddi/Docs/College/'
    try:
        path=os.path.join(parent_direct,directory)
        os.mkdir(path)
        print("Directory Created")
    except:
        pass
    source=open_file
    destination=path
    shutil.move(source,destination)
    print("File Moved")
    return destination

In [18]:
from tkinter import *
import PyPDF2
from tkinter import filedialog
import cv2 
import pytesseract
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
import nltk
from nltk.metrics.distance  import edit_distance
from nltk.corpus import stopwords
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import PIL.Image
import subprocess
import shlex

root = Tk()
root.title("PDF Reader!!")
root.geometry("200x200")

def clear_text_box():
    my_text.delete(1.0,END)
    
def open_pdf():
    open_file = filedialog.askopenfilename(
        initialdir="/Users/gauthammallipeddi/Docs/College/II-I/DBMS/",
        title = "Open PDF file",
        filetypes = (
        ("PDF Files","*.pdf"),
        ("All Files",".")))
    if open_file:
        page_stuff = pdf_word_extract(open_file)
        mydict=idf(page_stuff)
        Ans=findSubAndUnit(subcode_list,dataset,page_stuff,mydict)
        print(Ans)
        SubCode=Ans[0]
        Subject = FindSubName(SubCode)
        print(Subject)
        to_move = path_move(open_file,Subject)
        my_label.config(text=to_move)
        os.system("open"+shlex.quote(to_move))

my_button=Button(root,text="OPEN",command=open_pdf)
my_button.pack(pady=20)
my_label=Label(root,text=" ")
my_label.pack(pady=20)



root.mainloop()

['19bs1ph02', 1]
Engineering Physics
Directory Created
File Moved
